In [1]:
import faiss
from faiss import write_index, read_index
import polars as pl


In [2]:
test = pl.read_csv('test_dataset_submission_queries(1).csv')

In [3]:
search_cpu_index = read_index('solution/candidates.index')
search_cpu_index.is_trained, search_cpu_index.ntotal

(True, 34404561)

In [10]:
from sentence_transformers import SentenceTransformer
import torch

In [11]:
st_model = SentenceTransformer("inkoziev/sbert_pq", device=torch.device('cuda'))

In [12]:
queries =test['query'].to_list()

In [13]:
len(queries)

2000

In [14]:
qembeddings = None
st_model.eval()
with torch.no_grad():
    qembeddings = st_model.encode(
        queries,
        batch_size=40000,
        show_progress_bar=True
    )



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
_,faiss_ind = search_cpu_index.search(qembeddings, 10)

In [ ]:
del search_cpu_index

In [8]:
len(test['query'].unique())

2000

In [17]:
videos = pl.read_parquet('train_data/videos.parquet',columns=['video_id'])

In [26]:
videos['video_id'][faiss_ind[0]].to_list()

['video_3260098',
 'video_22926724',
 'video_23456026',
 'video_31701703',
 'video_17931383',
 'video_5865874',
 'video_14669863',
 'video_22356556',
 'video_31971374',
 'video_1809272']

In [29]:
result = {
    'query':[],
    'video_id':[]
}
for i in range(2000):
    result['video_id']+=videos['video_id'][faiss_ind[i]].to_list()
    result['query']+=[queries[i]]*10

In [30]:
res = pl.DataFrame(result)

In [23]:
res.write_csv('result.csv')

In [31]:
import catboost

In [32]:
from catboost import CatBoostRanker, Pool


In [34]:
model = CatBoostRanker().load_model('train_data/ranker.ckpt')

In [36]:
features = pl.read_parquet(
    'train_data/features.parquet',
)

In [38]:
videos = pl.read_parquet(
    'train_data/videos.parquet',
)

In [40]:
result1 = res.join(
    features,
    how='inner',
    on='video_id',
).join(
    videos,
    how='inner',
    on='video_id',
)
del features
del videos
result1 = result1.drop('report_date')

NameError: name 'automarkup' is not defined

In [41]:
result1 = result1.drop('report_date')

In [42]:
result1 = result1.with_columns(pl.col('v_channel_reg_datetime').str.to_datetime().cast(pl.Int64),pl.col('v_pub_datetime').str.to_datetime().cast(pl.Int64))\
    .drop('v_channel_reg_datetime_right','v_category_right','v_pub_datetime_right','v_channel_type_right','video_title')
                         

In [46]:
result1 = result1.sort('query').with_columns(pl.col('query').rank('min').alias('group_id'))


In [47]:
full_df = result1.to_pandas()

In [48]:
import pandas as pd
groups = pd.Series(full_df['group_id'].unique())


In [57]:
metainfo_columns = ['query', 'target','video_id', 'group_id']
to_pred=full_df.drop(['query','video_id', 'group_id'],axis=1)

In [72]:
metric = []
for i in range(2000):
    metric+=list(model.predict(to_pred.iloc[i*10:i*10+10]))

In [71]:
res = res.to_pandas()

In [73]:
res

,query,video_id
0,Битва сильнейших экстрасенсов 2023 смотреть | ...,video_3260098
1,Битва сильнейших экстрасенсов 2023 смотреть | ...,video_22926724
2,Битва сильнейших экстрасенсов 2023 смотреть | ...,video_23456026
3,Битва сильнейших экстрасенсов 2023 смотреть | ...,video_31701703
4,Битва сильнейших экстрасенсов 2023 смотреть | ...,video_17931383
...,...,...
19995,мужкое женское выпуск от 04 06 2021,video_21609247
19996,мужкое женское выпуск от 04 06 2021,video_27009675
19997,мужкое женское выпуск от 04 06 2021,video_32516801
19998,мужкое женское выпуск от 04 06 2021,video_10228398


In [74]:
res['range']=metric

In [80]:
res['group'] = res['query'].rank(method='min')

In [83]:
res

,query,video_id,range,group
0,Битва сильнейших экстрасенсов 2023 смотреть | ...,video_3260098,0.883286,1261.0
1,Битва сильнейших экстрасенсов 2023 смотреть | ...,video_22926724,0.883286,1261.0
2,Битва сильнейших экстрасенсов 2023 смотреть | ...,video_23456026,0.883286,1261.0
3,Битва сильнейших экстрасенсов 2023 смотреть | ...,video_31701703,0.883286,1261.0
4,Битва сильнейших экстрасенсов 2023 смотреть | ...,video_17931383,0.883286,1261.0
...,...,...,...,...
19995,мужкое женское выпуск от 04 06 2021,video_21609247,-0.097487,11201.0
19996,мужкое женское выпуск от 04 06 2021,video_27009675,-0.623009,11201.0
19997,мужкое женское выпуск от 04 06 2021,video_32516801,-0.148123,11201.0
19998,мужкое женское выпуск от 04 06 2021,video_10228398,0.452614,11201.0
